In [1]:
from skimage.io import imread
import glob
import os
import numpy as np

In [2]:
biosampleid='141'

In [3]:
datadir='/data/keerthi/brainpubdata/%s/BFIW_processed/' % biosampleid

In [4]:
secmin = {
    '222': 10,
    '141': 10,
    '142': 10,
    '213': 38,
    '244': 10,
}
    
secmax = {
    '222': 1730,
    '141': 1290,
    '142': 2100,
    '244': 2640,
    '213': 2610,
}

In [5]:
bfidict={}
for fn in glob.glob(datadir+'/*.jpg'):
    bn = os.path.basename(fn)
    secno = int(bn.split('_')[3])
    if secno >= secmin[biosampleid] and secno <= secmax[biosampleid]:
        bfidict[int(secno)]=fn

In [6]:
nsl = max(bfidict)
nsl

1290

In [7]:
imlast = imread(bfidict[nsl])

In [8]:
imlast.shape

(300, 475, 3)

In [9]:
arr = np.ones([nsl]+list(imlast.shape),imlast.dtype)*255

In [10]:
arr.shape

(1290, 300, 475, 3)

In [11]:
for slno in range(nsl):
    secno = slno+1
    if secno in bfidict:
        secarr = imread(bfidict[secno])
        arr[slno,...]=secarr

In [12]:
import SimpleITK as sitk

In [13]:
im = sitk.GetImageFromArray(arr)

In [14]:
im.SetSpacing([1.5,1.5,0.2])

In [15]:
drnmat=np.array([ [0,0,1], [-1,0,0], [0,-1,0] ]) # psr 

# sagittal LR
# rotmat = np.array([[-1,0,0],[0,-1,0],[0,0,1]]) # psr to asl

# sagittal RL
rotmat = np.eye(3) 

# coronal AP
if biosampleid=='213':
    rotmat = np.array([[0,-1,0],[1,0,0],[0,0,1]]) # psr to rsa

drn = np.dot(rotmat,drnmat).ravel()

In [16]:
im.SetDirection(drn.tolist())

In [17]:
os.makedirs('/data/keerthi/brainpubdata/%s/stack' % biosampleid,exist_ok=True)

In [18]:
sitk.WriteImage(im,'/data/keerthi/brainpubdata/%s/stack/%s_bfi_stack.mhd' % (biosampleid, biosampleid))